In [0]:
# PARAMETERS 1

from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# PARAMETERS 2

# Unmount the directory if it is already mounted
dbutils.fs.unmount("/mnt/project14")

# Define the configurations
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "?",
    "fs.azure.account.oauth2.client.secret": '?',
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/?/oauth2/token"
}

# Mount the directory
dbutils.fs.mount(
    source="abfss://?@?.dfs.core.windows.net", 
    mount_point="/mnt/project14",
    extra_configs=configs
)

/mnt/project14 has been unmounted.


True

In [0]:
#%fs
#ls "/mnt/project14"

In [0]:
# PARAMETERS 3

athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/project14/raw_data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/project14/raw_data/coaches.csv")
gender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/project14/raw_data/gender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/project14/raw_data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/mnt/project14/raw_data/teams.csv")

In [0]:
# TRANSFORMATION ATHLETES

from pyspark.sql.functions import col, split, concat_ws, initcap, expr

# Separar os nomes em uma array
athletes_transformed = athletes.withColumn("name_parts", split(col("PersonName"), " "))

# Concatenar corretamente o nome no formato "ÚltimoNome, PrimeiroNome1 PrimeiroNome2..."
athletes_transformed = athletes_transformed.withColumn("PersonName",
expr("concat_ws(', ', name_parts[size(name_parts)-1], array_join(slice(name_parts, 1, size(name_parts)-1), ' '))"))

# Aplicar Title Case (Primeira letra maiúscula, restante minúscula)
athletes_transformed = athletes_transformed.withColumn("PersonName", initcap(col("PersonName")))

# Remover a coluna auxiliar "name_parts"
athletes_transformed = athletes_transformed.drop("name_parts")

# Convert Spark DataFrame to Pandas DataFrame
athletes_transformed = athletes_transformed.toPandas()

# Salvar dataframe no Data Lake Gen 2
athletes_transformed.to_csv("/dbfs/mnt/project14/trans_data/athletes.csv", index=False)

# Mostrar o dataframe
display(athletes_transformed)


PersonName,Country,Discipline
"Katrine, Aalerud",Norway,Cycling Road
"Nestor, Abad",Spain,Artistic Gymnastics
"Giovanni, Abagnale",Italy,Rowing
"Alberto, Abalde",Spain,Basketball
"Tamara, Abalde",Spain,Basketball
"Luc, Abalo",France,Handball
"Cesar, Abaroa",Chile,Rowing
"Abobakr, Abass",Sudan,Swimming
"Hamideh, Abbasali",Islamic Republic of Iran,Karate
"Islam, Abbasov",Azerbaijan,Wrestling


In [0]:
# TRANSFORMATION COACHES

from pyspark.sql.functions import col, split, concat_ws, initcap, expr

# Separar os nomes em uma array
coaches_transformed = coaches.withColumn("name_parts", split(col("Name"), " "))

# Concatenar corretamente o nome no formato "ÚltimoNome, PrimeiroNome1 PrimeiroNome2..."
coaches_transformed = coaches_transformed.withColumn("Name",
expr("concat_ws(', ', name_parts[size(name_parts)-1], array_join(slice(name_parts, 1, size(name_parts)-1), ' '))"))

# Aplicar Title Case (Primeira letra maiúscula, restante minúscula)
coaches_transformed = coaches_transformed.withColumn("Name", initcap(col("Name")))

# Remover a coluna auxiliar "name_parts"
coaches_transformed = coaches_transformed.drop("name_parts")

# Convert Spark DataFrame to Pandas DataFrame
coaches_transformed = coaches_transformed.toPandas()

# Salvar dataframe no Data Lake Gen 2
coaches_transformed.to_csv("/dbfs/mnt/project14/trans_data/coaches.csv", index=False)

# Mostrar o dataframe
display(coaches_transformed)

Name,Country,Discipline,Event
"Wael, Abdelmagid",Egypt,Football,null
"Junya, Abe",Japan,Volleyball,null
"Katsuhiko, Abe",Japan,Basketball,null
"Cherif, Adama",C�te d'Ivoire,Football,null
"Yuya, Ageba",Japan,Volleyball,null
"Gottlieb, Aikman Siegfried",Japan,Hockey,Men
"Kais, Al Saadi",Germany,Hockey,Men
"Lonni, Alameda",Canada,Baseball/Softball,Softball
"Vladimir, Alekno",Islamic Republic of Iran,Volleyball,Men
"Alexey, Alekseev",ROC,Handball,Women


In [0]:
# TRANSFORMATION GENDER

from pyspark.sql.functions import col, rank, expr
from pyspark.sql.window import Window

# Convert Spark DataFrame to Pandas DataFrame
gender_transformed = gender.toPandas()

# Salvar dataframe no Data Lake Gen 2
gender_transformed.to_csv("/dbfs/mnt/project14/trans_data/gender.csv", index=False)

# Mostrar o dataframe
display(gender_transformed)


Discipline,Female,Male,Total
3x3 Basketball,32,32,64
Archery,64,64,128
Artistic Gymnastics,98,98,196
Artistic Swimming,105,0,105
Athletics,969,1072,2041
Badminton,86,87,173
Baseball/Softball,90,144,234
Basketball,144,144,288
Beach Volleyball,48,48,96
Boxing,102,187,289


In [0]:
# TRANSFORMATION TEAMS

# Remover a coluna "TeamName"
teams_transformed = teams.drop("TeamName")

# Convert Spark DataFrame to Pandas DataFrame
teams_transformed = teams_transformed.toPandas()

# Salvar dataframe no Data Lake Gen 2
teams_transformed.to_csv("/dbfs/mnt/project14/trans_data/teams.csv", index=False)

# Mostrar o dataframe
display(teams_transformed)


Discipline,Country,Event
3x3 Basketball,Belgium,Men
3x3 Basketball,People's Republic of China,Men
3x3 Basketball,People's Republic of China,Women
3x3 Basketball,France,Women
3x3 Basketball,Italy,Women
3x3 Basketball,Japan,Men
3x3 Basketball,Japan,Women
3x3 Basketball,Latvia,Men
3x3 Basketball,Mongolia,Women
3x3 Basketball,Netherlands,Men


In [0]:
# TRANSFORMATION MEDALS

from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank

# Convert Spark DataFrame to Pandas DataFrame
medals_transformed = medals.toPandas()

# Salvar dataframe no Data Lake Gen 2
medals_transformed.to_csv("/dbfs/mnt/project14/trans_data/medals.csv", index=False)

# Mostrar o dataframe
display(medals_transformed)


Rank,Team_Country,Gold,Silver,Bronze,Total,Rank by Total
1,United States of America,39,41,33,113,1
2,People's Republic of China,38,32,18,88,2
3,Japan,27,14,17,58,5
4,Great Britain,22,21,22,65,4
5,ROC,20,28,23,71,3
6,Australia,17,7,22,46,6
7,Netherlands,10,12,14,36,9
8,France,10,12,11,33,10
9,Germany,10,11,16,37,8
10,Italy,10,10,20,40,7
